# dbachecks - Configurations

You can use dbachecks on the command line as we have seen in the previous notebook but you will not always want to have the default settings for the checks

You can alter any of the configurations using the `Set-DbcConfig` command.

## Invalid Database Owners

You may want to check that databases are not owned by certain accounts.

By default the `InvalidDatabaseOwner` checks that the databases are not owned by the sa account. When we run this check against the instance we pass all of the checks


Again, this requires that you have the containers set up from the Notebook 00 - Setting up the containers for the rest of the containers.ipynb

If you have altered the folder path you will need to alter it below.


In [23]:
$FolderPath = $Env:USERPROFILE + '\Documents\dbachecks'
$SqlInstances = 'localhost,15592','localhost,15593'
$SqlCredential = Import-Clixml -Path $FolderPath\sqladmin.cred
# Just in case the config has been altered
$null = Reset-DbcConfig
Invoke-DbcCheck -SqlInstance $SqlInstances -SqlCredential $SqlCredential  InvalidDatabaseOwner

    ____            __
   / __ \___  _____/ /____  _____
  / /_/ / _ \/ ___/ __/ _ \/ ___/
 / ____/  __(__  ) /_/  __/ /
/_/    \___/____/\__/\___/_/
Pester v4.9.0
Executing all tests in 'C:\Program Files\WindowsPowerShell\Modules\dbachecks\1.2.21\checks\Database.Tests.ps1' with Tags InvalidDatabaseOwner

Executing script C:\Program Files\WindowsPowerShell\Modules\dbachecks\1.2.21\checks\Database.Tests.ps1

  Describing Invalid Database Owner

    Context Testing Database Owners on localhost,15592
      [+] Database AdventureWorks2017 - owner sqladmin should Not be in this list ( sa ) on localhost,15592 1ms
      [+] Database Northwind - owner sqladmin should Not be in this list ( sa ) on localhost,15592 1ms
      [+] Database pubs - owner sqladmin should Not be in this list ( sa ) on localhost,15592 1ms
      [+] Database WideWorldImporters - owner sqladmin should Not be in this list ( sa ) on localhost,15592 2ms

  Describing Invalid Database Owner

    Context Testing Database Owner

If our organistation requirements are that no databases should be owned by members of the DBA Team user accounts, we can alter the configuration to check for that. 

To find out which configuration we need to change we can use `Get-DbcCheck`

In [5]:
Get-DbcCheck -Pattern InvalidDatabaseOwner | Format-List



Group       : Database
Type        : Sqlinstance
UniqueTag   : InvalidDatabaseOwner
AllTags     : InvalidDatabaseOwner, Medium, Database
Config      : app.sqlinstance policy.invaliddbowner.name policy.invaliddbowner.excludedb 
Description : Tests that the Database Owner is NOT in the specified (default blank) list





We can see that there are 3 configurations that this check will use
- app.sqlinstance
- policy.invaliddbowner.name 
- policy.invaliddbowner.excludedb 

We can get further information about the checks with `Get-DbcConfig`

In [6]:
Get-DbcConfig -Name policy.invaliddbowner.name | Format-List



Name        : policy.invaliddbowner.name
Value       : sa
Description : The database owner account should not be this user





We can change the value of that configuration item using `Set-DbcConfig`. We will set it to the members of the DBA Team AD Group

This is an example - don't run this unless you are connected to THEBEARD.local domain :-)

In [19]:
$DBA_Accounts =  (Get-ADGroup -Identity 'CN=SQL_DBAs_The_Cool_Ones,OU=TheBeardGroups,OU=TheBeard,DC=TheBeard,DC=local' |Get-ADGroupMember).ForEach{"THEBEARD\$($Psitem.SamAccountName)"} 
Set-DbcConfig -Name policy.invaliddbowner.name -Value $DBA_Accounts


Name                       Value                                                                Description            
----                       -----                                                                -----------            
policy.invaliddbowner.name {THEBEARD\Rob, THEBEARD\DBA, THEBEARD\wdurkin, THEBEARD\gsartori...} The database owner a...




## This is how you can alter a check to test the values that YOU want by setting a configuration
We have disabled the sa acccount and disabled the old account - Perhaps we do not want our databases owned by the new sysadmin account `sqladmin` We can set that value in the configuration and run the test

In [24]:
$FolderPath = $Env:USERPROFILE + '\Documents\dbachecks'
$SqlInstances = 'localhost,15592','localhost,15593'
$SqlCredential = Import-Clixml -Path $FolderPath\sqladmin.cred
Set-DbcConfig -Name policy.invaliddbowner.name -Value sqladmin
Invoke-DbcCheck -SqlInstance $SqlInstances -SqlCredential $SqlCredential -Check InvalidDatabaseOwner


    ____            __
   / __ \___  _____/ /____  _____
  / /_/ / _ \/ ___/ __/ _ \/ ___/
 / ____/  __(__  ) /_/  __/ /
/_/    \___/____/\__/\___/_/
Pester v4.9.0
Executing all tests in 'C:\Program Files\WindowsPowerShell\Modules\dbachecks\1.2.21\checks\Database.Tests.ps1' with Tags InvalidDatabaseOwner

Executing script C:\Program Files\WindowsPowerShell\Modules\dbachecks\1.2.21\checks\Database.Tests.ps1

  Describing Invalid Database Owner

    Context Testing Database Owners on localhost,15592
      [-] Database AdventureWorks2017 - owner sqladmin should Not be in this list ( sqladmin ) on localhost,15592 3ms
        Expected collection sqladmin to not contain 'sqladmin', because The database owner was one specified as incorrect, but it was found.
        176:                         $psitem.Owner | Should -Not -BeIn $TargetOwner -Because "The database owner was one specified as incorrect"
        at <ScriptBlock>, C:\Program Files\WindowsPowerShell\Modules\dbachecks\1.2.21\checks

Ah

We had better have a word with that fellow Rob and alter all of those database owners!! 

## This is how we could ask our junior DBA to do this and test they have done so correctly

We could change the owner with `Get-DbaDatabase -SqlInstance sql2016n1 -ExcludeSystem -Owner THEBEARD\Rob | Set-DbaDbOwner -TargetLogin THEBEARD\DatabaseOwner`

We can also set a confiuguration for our desired database owner and test for that. With dbachecks we can check for both Valid and Invalid database owner accounts



In [25]:
$FolderPath = $Env:USERPROFILE + '\Documents\dbachecks'
$SqlInstances = 'localhost,15592','localhost,15593'
$SqlCredential = Import-Clixml -Path $FolderPath\sqladmin.cred
# Set the correct owner
$null = Get-DbaDatabase -SqlInstance $SqlInstances -SqlCredential $SqlCredential -ExcludeSystem -Owner sqladmin | Set-DbaDbOwner -TargetLogin thebeard
# Check that we do not have any databases owned by the unexpected accounts
Invoke-DbcCheck -SqlInstance $SqlInstances -SqlCredential $SqlCredential -Check InvalidDatabaseOwner
# Set the value for expected database owner
Set-DbcConfig -Name policy.validdbowner.name -Value thebeard
# Check our databases  are owned by the correct account
Invoke-DbcCheck -SqlInstance $SqlInstances -SqlCredential $SqlCredential -Check ValidDatabaseOwner 

    ____            __
   / __ \___  _____/ /____  _____
  / /_/ / _ \/ ___/ __/ _ \/ ___/
 / ____/  __(__  ) /_/  __/ /
/_/    \___/____/\__/\___/_/
Pester v4.9.0
Executing all tests in 'C:\Program Files\WindowsPowerShell\Modules\dbachecks\1.2.21\checks\Database.Tests.ps1' with Tags InvalidDatabaseOwner

Executing script C:\Program Files\WindowsPowerShell\Modules\dbachecks\1.2.21\checks\Database.Tests.ps1

  Describing Invalid Database Owner

    Context Testing Database Owners on localhost,15592
      [+] Database AdventureWorks2017 - owner thebeard should Not be in this list ( sqladmin ) on localhost,15592 1ms
      [+] Database Northwind - owner thebeard should Not be in this list ( sqladmin ) on localhost,15592 2ms
      [+] Database pubs - owner thebeard should Not be in this list ( sqladmin ) on localhost,15592 1ms
      [+] Database WideWorldImporters - owner thebeard should Not be in this list ( sqladmin ) on localhost,15592 1ms

  Describing Invalid Database Owner

    Contex

## Ensuring Databases exist

You might want to test that a particular database exists on your instance. This could be your DBA database or perhaps you have a check for your companies software system that you want to run on a clients estate and your software requires certain databases to exist 

You can run the default `DatabaseExists` which will check that the system databases exist as we showed in the quick examples notebook.

In [26]:
$FolderPath = $Env:USERPROFILE + '\Documents\dbachecks'
$SqlInstances = 'localhost,15592','localhost,15593'
$SqlCredential = Import-Clixml -Path $FolderPath\sqladmin.cred
Set-DbcConfig -Name policy.invaliddbowner.name -Value sqladmin
Invoke-DbcCheck -SqlInstance $SqlInstances -SqlCredential $SqlCredential -Check DatabaseExists


    ____            __
   / __ \___  _____/ /____  _____
  / /_/ / _ \/ ___/ __/ _ \/ ___/
 / ____/  __(__  ) /_/  __/ /
/_/    \___/____/\__/\___/_/
Pester v4.9.0
Executing all tests in 'C:\Program Files\WindowsPowerShell\Modules\dbachecks\1.2.21\checks\Database.Tests.ps1' with Tags DatabaseExists

Executing script C:\Program Files\WindowsPowerShell\Modules\dbachecks\1.2.21\checks\Database.Tests.ps1

  Describing Database Exists

    Context Database exists on localhost,15592
      [+] Database master should exist on  82ms
      [+] Database msdb should exist on  77ms
      [+] Database tempdb should exist on  76ms
      [+] Database model should exist on  79ms

  Describing Database Exists

    Context Database exists on localhost,15593
      [+] Database master should exist on  80ms
      [+] Database msdb should exist on  84ms
      [+] Database tempdb should exist on  85ms
      [+] Database model should exist on  81ms
Tests completed in 1.03s
Tests Passed: 8, Failed: 0, Skipped:

### Finding the configurations for the check

You can find the configurations that you can set for a check with `Get-DbcCheck`

In [12]:
Get-DbcCheck DatabaseExists | Format-List



Group       : Database
Type        : Sqlinstance
UniqueTag   : DatabaseExists
AllTags     : DatabaseExists, Database
Config      : app.sqlinstance database.exists 
Description : Tests that the databases are specified are on the instance (Note - Does not check if they are available 
              - Use DatabaseStatus for that))





You can see that you can set the configuration `database.exists` Note that we are using the `-Append` parameter to add to the configuration instead of replacing it.

In [27]:
Set-DbcConfig -Name database.exists -Value 'pubs','NorthWind','AdventureWorks2017','WideWorldImporters' -Append


Name            Value                            Description                                   
----            -----                            -----------                                   
database.exists {master, msdb, tempdb, model...} The databases we expect to be on the instances




and then we can run the check again and confirm that we have all of the expected databases.

In [28]:
$FolderPath = $Env:USERPROFILE + '\Documents\dbachecks'
$SqlInstances = 'localhost,15592','localhost,15593'
$SqlCredential = Import-Clixml -Path $FolderPath\sqladmin.cred
Set-DbcConfig -Name policy.invaliddbowner.name -Value sqladmin
Invoke-DbcCheck -SqlInstance $SqlInstances -SqlCredential $SqlCredential -Check DatabaseExists


    ____            __
   / __ \___  _____/ /____  _____
  / /_/ / _ \/ ___/ __/ _ \/ ___/
 / ____/  __(__  ) /_/  __/ /
/_/    \___/____/\__/\___/_/
Pester v4.9.0
Executing all tests in 'C:\Program Files\WindowsPowerShell\Modules\dbachecks\1.2.21\checks\Database.Tests.ps1' with Tags DatabaseExists

Executing script C:\Program Files\WindowsPowerShell\Modules\dbachecks\1.2.21\checks\Database.Tests.ps1

  Describing Database Exists

    Context Database exists on localhost,15592
      [+] Database master should exist on  79ms
      [+] Database msdb should exist on  77ms
      [+] Database tempdb should exist on  76ms
      [+] Database model should exist on  84ms
      [+] Database pubs should exist on  78ms
      [+] Database NorthWind should exist on  82ms
      [+] Database AdventureWorks2017 should exist on  75ms
      [+] Database WideWorldImporters should exist on  82ms

  Describing Database Exists

    Context Database exists on localhost,15593
      [+] Database master should 

## Ensuring the Agent Alerts are as expected

You might want to check that you have all of the expected Agent Alerts. By default dbachecks will check for alerts for Severity 16-25 and messageid 823,824,825

In [31]:
$FolderPath = $Env:USERPROFILE + '\Documents\dbachecks'
$SqlInstances = 'localhost,15592','localhost,15593'
$SqlCredential = Import-Clixml -Path $FolderPath\sqladmin.cred
Invoke-DbcCheck -SqlInstance $SqlInstances -SqlCredential $SqlCredential -Check AgentAlert

    ____            __
   / __ \___  _____/ /____  _____
  / /_/ / _ \/ ___/ __/ _ \/ ___/
 / ____/  __(__  ) /_/  __/ /
/_/    \___/____/\__/\___/_/
Pester v4.9.0
Executing all tests in 'C:\Program Files\WindowsPowerShell\Modules\dbachecks\1.2.21\checks\Agent.Tests.ps1' with Tags AgentAlert

Executing script C:\Program Files\WindowsPowerShell\Modules\dbachecks\1.2.21\checks\Agent.Tests.ps1

  Describing Agent Alerts

    Context Testing Agent Alerts Severity exists on localhost,15592
      [+] localhost,15592 should have Severity 16 Alert 2ms
      [+] localhost,15592 should have Severity 16 Alert enabled 1ms
      [+] localhost,15592 should have notification for Severity 16 Alert 1ms
      [+] localhost,15592 should have Severity 17 Alert 2ms
      [+] localhost,15592 should have Severity 17 Alert enabled 2ms
      [+] localhost,15592 should have notification for Severity 17 Alert 3ms
      [+] localhost,15592 should have Severity 18 Alert 1ms
      [+] localhost,15592 should have Se

You can check the config items available for the check

In [33]:
Get-DbcCheck -Pattern AgentAlert | Format-List



Group       : Agent
Type        : Sqlinstance
UniqueTag   : AgentAlert
AllTags     : AgentAlert, Agent
Config      : app.sqlinstance agent.alert.Severity agent.alert.messageid agent.alert.Job agent.alert.Notification 
Description : Tests that there are Agent Alerts set up for the specified (default 16-25) alert severities and ids 
              (default 823-825) and if specified Agent Jobs and/or notifications





The configuration items available are
- app.sqlinstance 
- agent.alert.Severity 
- agent.alert.messageid 
- agent.alert.Job 
- agent.alert.Notification



In [37]:
'app.sqlinstance', 'agent.alert.Severity', 'agent.alert.messageid', 'agent.alert.Job', 'agent.alert.Notification' | ForEach-Object {
    Get-DbcConfig -Name $PSItem | Format-List
}



Name        : app.sqlinstance
Value       : 
Description : List of SQL Server instances that SQL-based tests will run against





Name        : agent.alert.Severity
Value       : {16, 17, 18, 19...}
Description : Agent alert severity to validate; https://www.brentozar.com/blitz/configure-sql-server-alerts/





Name        : agent.alert.messageid
Value       : {823, 824, 825}
Description : Agent alert messageid to validate; https://www.brentozar.com/blitz/configure-sql-server-alerts/





Name        : agent.alert.Job
Value       : False
Description : Agent alert job notification. Ex job to write to eventlog for SCOM monitoring





Name        : agent.alert.Notification
Value       : True
Description : Agent alert notification



